# Introduccion y objetivo
Este trabajo se centra en detectar segun imagenes el cáncer de piel de tipo melanoma. Esta es una de las enfermedades mas graves en dermatologia, y la deteccion temprana a traves de imagenes es clave para reducir riesgos y mejorar tratamientos. El objetivo es entrenar un modelo de clasifiacion de imagenes capaz de distinguir entre melanoma y no melanoma, exporando como distintos hiperparametros (como la tasa de aprendizaje, numero de epocas, tamaño de batch, regularizacion, entre otros) afectan el desempeño. La metrica principal para evaluar es el F1 Score.

# EDA

In [ ]:
import pandas as pd
from pathlib import Path

# Rutas Archivos

train_mel   = Path(r"C:\Users\nonit\Desktop\Detección Melanoma\data\train\mel")
train_nomel = Path(r"C:\Users\nonit\Desktop\Detección Melanoma\data\train\nomel")
val_mel     = Path(r"C:\Users\nonit\Desktop\Detección Melanoma\data\val\mel")
val_nomel   = Path(r"C:\Users\nonit\Desktop\Detección Melanoma\data\val\nomel")
test_path   = Path(r"C:\Users\nonit\Desktop\Detección Melanoma\data\test")

# Archivos CSV
test_csv = pd.read_csv(r"C:\Users\nonit\Desktop\Detección Melanoma\data\test.csv", encoding="utf-8")

# Conteo
def list_images(path):
    return [f for f in path.glob("*") if f.suffix.lower() in [".jpg", ".jpeg", ".png"]]

train_mel_imgs   = list_images(train_mel)
train_nomel_imgs = list_images(train_nomel)
val_mel_imgs     = list_images(val_mel)
val_nomel_imgs   = list_images(val_nomel)
test_imgs        = list_images(test_path)

# Resumen

print("___________ TRAIN ___________")
print(f"Melanoma    : {len(train_mel_imgs)}")
print(f"No Melanoma : {len(train_nomel_imgs)}")
print(f"Total       : {len(train_mel_imgs) + len(train_nomel_imgs)}")

print("___________ VALIDATION ___________")
print(f"Melanoma    : {len(val_mel_imgs)}")
print(f"No Melanoma : {len(val_nomel_imgs)}")
print(f"Total       : {len(val_mel_imgs) + len(val_nomel_imgs)}")

print("___________ TEST ___________")
print(f"Total imágenes en carpeta test : {len(test_imgs)}")
print(f"Total IDs en test.csv          : {len(test_csv)}")


# Tipo de archivos

ext_train = set([img.suffix.lower() for img in train_mel_imgs + train_nomel_imgs])
ext_val   = set([img.suffix.lower() for img in val_mel_imgs + val_nomel_imgs])
ext_test  = set([img.suffix.lower() for img in test_imgs])

print("\n _______ FORMATOS _______")
print(f"Train: {ext_train}")
print(f"Val  : {ext_val}")
print(f"Test : {ext_test}")
